In [1]:
# train with fewer data. ideal data
import keras
import pandas as pd
import numpy as np
from keras import layers
from keras import regularizers
from sklearn import preprocessing
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, SGD, Adam
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model, load_model
from keras.callbacks import ModelCheckpoint, Callback
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_probability as tfp

2023-02-23 12:47:55.378038: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-23 12:47:55.502167: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-23 12:47:55.502226: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-23 12:47:56.168924: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
e_data = pd.read_csv("data.csv")
label = pd.read_csv("properity.csv")

conditions = ["materials", "color", "distance", "mode"]

In [3]:
#r_data = r_data.drop(columns = conditions)
#r_data = pd.concat([label, r_data], axis = 1)

e_data = e_data.drop(columns = conditions)
e_data = pd.concat([label, e_data], axis = 1)

e_data = e_data.drop(columns = ["Hardness", "Rigidity", "Strength"])

In [4]:
e_data

,Density,names,spectrum_0,spectrum_1,spectrum_2,spectrum_3,spectrum_4,spectrum_5,spectrum_6,spectrum_7,...,spectrum_321,spectrum_322,spectrum_323,spectrum_324,spectrum_325,spectrum_326,spectrum_327,spectrum_328,spectrum_329,spectrum_330
0,0.7,Cardboard,0.526403,0.527891,0.529310,0.530688,0.532068,0.533471,0.534901,0.536361,...,0.670099,0.670870,0.671219,0.671284,0.671310,0.671274,0.671686,0.672272,0.672768,0.672819
1,0.7,Cardboard,0.542235,0.543597,0.544961,0.546345,0.547773,0.549250,0.550766,0.552315,...,0.685934,0.686723,0.687080,0.687148,0.687176,0.687140,0.687564,0.688165,0.688675,0.688728
2,0.7,Cardboard,0.536953,0.538290,0.539612,0.540949,0.542335,0.543779,0.545277,0.546822,...,0.679368,0.680120,0.680447,0.680490,0.680496,0.680442,0.680844,0.681425,0.681916,0.681958
3,0.7,Cardboard,0.543379,0.544370,0.545400,0.546501,0.547699,0.549000,0.550391,0.551859,...,0.686358,0.687150,0.687509,0.687577,0.687605,0.687568,0.687990,0.688591,0.689099,0.689151
4,0.7,Cardboard,0.523413,0.524326,0.525328,0.526431,0.527640,0.528946,0.530326,0.531762,...,0.661900,0.662690,0.663060,0.663147,0.663193,0.663175,0.663598,0.664191,0.664694,0.664755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,0.8,Wood,0.501132,0.501212,0.501278,0.501364,0.501494,0.501661,0.501850,0.502045,...,0.543924,0.544602,0.544929,0.545019,0.545072,0.545068,0.545425,0.545920,0.546338,0.546392
1256,0.8,Wood,0.462299,0.462222,0.462148,0.462112,0.462131,0.462201,0.462306,0.462434,...,0.500846,0.501468,0.501766,0.501847,0.501893,0.501888,0.502214,0.502668,0.503051,0.503099
1257,0.8,Wood,0.197083,0.197172,0.197252,0.197331,0.197412,0.197492,0.197564,0.197624,...,0.216179,0.216449,0.216579,0.216614,0.216634,0.216632,0.216773,0.216968,0.217133,0.217154
1258,0.8,Wood,0.183714,0.183832,0.183919,0.183990,0.184056,0.184118,0.184174,0.184224,...,0.200452,0.200693,0.200805,0.200831,0.200843,0.200835,0.200960,0.201136,0.201285,0.201301


In [5]:
# normalize data 
tep = e_data.loc[:, [ "names", "Density"]]
e_data_sp = e_data.copy().drop(columns = ['names', "Density"])
e_data_sp

,spectrum_0,spectrum_1,spectrum_2,spectrum_3,spectrum_4,spectrum_5,spectrum_6,spectrum_7,spectrum_8,spectrum_9,...,spectrum_321,spectrum_322,spectrum_323,spectrum_324,spectrum_325,spectrum_326,spectrum_327,spectrum_328,spectrum_329,spectrum_330
0,0.526403,0.527891,0.529310,0.530688,0.532068,0.533471,0.534901,0.536361,0.537855,0.539371,...,0.670099,0.670870,0.671219,0.671284,0.671310,0.671274,0.671686,0.672272,0.672768,0.672819
1,0.542235,0.543597,0.544961,0.546345,0.547773,0.549250,0.550766,0.552315,0.553888,0.555468,...,0.685934,0.686723,0.687080,0.687148,0.687176,0.687140,0.687564,0.688165,0.688675,0.688728
2,0.536953,0.538290,0.539612,0.540949,0.542335,0.543779,0.545277,0.546822,0.548405,0.550009,...,0.679368,0.680120,0.680447,0.680490,0.680496,0.680442,0.680844,0.681425,0.681916,0.681958
3,0.543379,0.544370,0.545400,0.546501,0.547699,0.549000,0.550391,0.551859,0.553386,0.554947,...,0.686358,0.687150,0.687509,0.687577,0.687605,0.687568,0.687990,0.688591,0.689099,0.689151
4,0.523413,0.524326,0.525328,0.526431,0.527640,0.528946,0.530326,0.531762,0.533235,0.534721,...,0.661900,0.662690,0.663060,0.663147,0.663193,0.663175,0.663598,0.664191,0.664694,0.664755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,0.501132,0.501212,0.501278,0.501364,0.501494,0.501661,0.501850,0.502045,0.502237,0.502413,...,0.543924,0.544602,0.544929,0.545019,0.545072,0.545068,0.545425,0.545920,0.546338,0.546392
1256,0.462299,0.462222,0.462148,0.462112,0.462131,0.462201,0.462306,0.462434,0.462574,0.462712,...,0.500846,0.501468,0.501766,0.501847,0.501893,0.501888,0.502214,0.502668,0.503051,0.503099
1257,0.197083,0.197172,0.197252,0.197331,0.197412,0.197492,0.197564,0.197624,0.197671,0.197703,...,0.216179,0.216449,0.216579,0.216614,0.216634,0.216632,0.216773,0.216968,0.217133,0.217154
1258,0.183714,0.183832,0.183919,0.183990,0.184056,0.184118,0.184174,0.184224,0.184265,0.184299,...,0.200452,0.200693,0.200805,0.200831,0.200843,0.200835,0.200960,0.201136,0.201285,0.201301


In [6]:
e_data_sp = (e_data_sp - e_data_sp.mean())/e_data_sp.std()
e_data_sp

,spectrum_0,spectrum_1,spectrum_2,spectrum_3,spectrum_4,spectrum_5,spectrum_6,spectrum_7,spectrum_8,spectrum_9,...,spectrum_321,spectrum_322,spectrum_323,spectrum_324,spectrum_325,spectrum_326,spectrum_327,spectrum_328,spectrum_329,spectrum_330
0,0.058279,0.063007,0.067694,0.072298,0.076858,0.081422,0.086024,0.090695,0.095454,0.100302,...,0.557489,0.557749,0.557980,0.558187,0.558372,0.558535,0.558680,0.558808,0.558920,0.559013
1,0.107668,0.112021,0.116560,0.121207,0.125941,0.130761,0.135658,0.140626,0.145654,0.150725,...,0.610426,0.610694,0.610936,0.611154,0.611349,0.611523,0.611679,0.611817,0.611937,0.612038
2,0.091189,0.095456,0.099860,0.104353,0.108945,0.113654,0.118484,0.123433,0.128489,0.133625,...,0.588477,0.588641,0.588790,0.588924,0.589045,0.589153,0.589249,0.589335,0.589409,0.589472
3,0.111236,0.114431,0.117930,0.121694,0.125711,0.129980,0.134483,0.139196,0.144083,0.149092,...,0.611844,0.612121,0.612367,0.612586,0.612779,0.612950,0.613102,0.613235,0.613350,0.613446
4,0.048949,0.051882,0.055263,0.058999,0.063017,0.067271,0.071713,0.076300,0.080989,0.085738,...,0.530075,0.530425,0.530738,0.531018,0.531267,0.531489,0.531685,0.531859,0.532009,0.532135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,-0.020559,-0.020252,-0.019827,-0.019304,-0.018699,-0.018044,-0.017370,-0.016704,-0.016065,-0.015464,...,0.135658,0.136021,0.136337,0.136612,0.136852,0.137061,0.137242,0.137398,0.137532,0.137642
1256,-0.141702,-0.141929,-0.141998,-0.141924,-0.141726,-0.141434,-0.141075,-0.140675,-0.140253,-0.139825,...,-0.008360,-0.008045,-0.007770,-0.007532,-0.007325,-0.007146,-0.006990,-0.006856,-0.006743,-0.006649
1257,-0.969075,-0.969072,-0.969058,-0.969057,-0.969083,-0.969153,-0.969274,-0.969449,-0.969673,-0.969936,...,-0.960065,-0.959989,-0.959924,-0.959869,-0.959823,-0.959784,-0.959750,-0.959722,-0.959699,-0.959680
1258,-1.010783,-1.010703,-1.010686,-1.010730,-1.010826,-1.010971,-1.011161,-1.011389,-1.011647,-1.011926,...,-1.012643,-1.012611,-1.012586,-1.012566,-1.012551,-1.012540,-1.012531,-1.012525,-1.012520,-1.012517


In [7]:
e_data = pd.concat([tep, e_data_sp], axis =1)
e_data

,names,Density,spectrum_0,spectrum_1,spectrum_2,spectrum_3,spectrum_4,spectrum_5,spectrum_6,spectrum_7,...,spectrum_321,spectrum_322,spectrum_323,spectrum_324,spectrum_325,spectrum_326,spectrum_327,spectrum_328,spectrum_329,spectrum_330
0,Cardboard,0.7,0.058279,0.063007,0.067694,0.072298,0.076858,0.081422,0.086024,0.090695,...,0.557489,0.557749,0.557980,0.558187,0.558372,0.558535,0.558680,0.558808,0.558920,0.559013
1,Cardboard,0.7,0.107668,0.112021,0.116560,0.121207,0.125941,0.130761,0.135658,0.140626,...,0.610426,0.610694,0.610936,0.611154,0.611349,0.611523,0.611679,0.611817,0.611937,0.612038
2,Cardboard,0.7,0.091189,0.095456,0.099860,0.104353,0.108945,0.113654,0.118484,0.123433,...,0.588477,0.588641,0.588790,0.588924,0.589045,0.589153,0.589249,0.589335,0.589409,0.589472
3,Cardboard,0.7,0.111236,0.114431,0.117930,0.121694,0.125711,0.129980,0.134483,0.139196,...,0.611844,0.612121,0.612367,0.612586,0.612779,0.612950,0.613102,0.613235,0.613350,0.613446
4,Cardboard,0.7,0.048949,0.051882,0.055263,0.058999,0.063017,0.067271,0.071713,0.076300,...,0.530075,0.530425,0.530738,0.531018,0.531267,0.531489,0.531685,0.531859,0.532009,0.532135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,Wood,0.8,-0.020559,-0.020252,-0.019827,-0.019304,-0.018699,-0.018044,-0.017370,-0.016704,...,0.135658,0.136021,0.136337,0.136612,0.136852,0.137061,0.137242,0.137398,0.137532,0.137642
1256,Wood,0.8,-0.141702,-0.141929,-0.141998,-0.141924,-0.141726,-0.141434,-0.141075,-0.140675,...,-0.008360,-0.008045,-0.007770,-0.007532,-0.007325,-0.007146,-0.006990,-0.006856,-0.006743,-0.006649
1257,Wood,0.8,-0.969075,-0.969072,-0.969058,-0.969057,-0.969083,-0.969153,-0.969274,-0.969449,...,-0.960065,-0.959989,-0.959924,-0.959869,-0.959823,-0.959784,-0.959750,-0.959722,-0.959699,-0.959680
1258,Wood,0.8,-1.010783,-1.010703,-1.010686,-1.010730,-1.010826,-1.010971,-1.011161,-1.011389,...,-1.012643,-1.012611,-1.012586,-1.012566,-1.012551,-1.012540,-1.012531,-1.012525,-1.012520,-1.012517


In [8]:
test_list = ["Cardboard1", "Ceramic1", "Glass1", "Plastic1", "Rock1"]
e_test = e_data[(e_data["names"]=="Cardboard1" )|(e_data["names"]=="Ceramic1" )|(e_data["names"]=="Glass1") |(e_data["names"]=="Plastic1") |(e_data["names"]=="Rock1")].reset_index(drop=True)
e_test

,names,Density,spectrum_0,spectrum_1,spectrum_2,spectrum_3,spectrum_4,spectrum_5,spectrum_6,spectrum_7,...,spectrum_321,spectrum_322,spectrum_323,spectrum_324,spectrum_325,spectrum_326,spectrum_327,spectrum_328,spectrum_329,spectrum_330
0,Cardboard1,0.7,-0.028709,-0.025573,-0.022161,-0.018525,-0.014682,-0.010642,-0.006427,-0.002061,...,0.599317,0.599639,0.599922,0.600171,0.600390,0.600583,0.600752,0.600899,0.601027,0.601132
1,Cardboard1,0.7,0.005553,0.009694,0.013887,0.018083,0.022298,0.026552,0.030853,0.035206,...,0.650668,0.651063,0.651414,0.651725,0.652000,0.652243,0.652458,0.652646,0.652808,0.652944
2,Cardboard1,0.7,0.038184,0.042790,0.047380,0.051909,0.056409,0.060903,0.065401,0.069906,...,0.700868,0.701222,0.701535,0.701811,0.702056,0.702271,0.702461,0.702627,0.702771,0.702891
3,Cardboard1,0.7,-0.464907,-0.462942,-0.460760,-0.458399,-0.455878,-0.453210,-0.450416,-0.447522,...,-0.028375,-0.027961,-0.027591,-0.027261,-0.026967,-0.026707,-0.026477,-0.026274,-0.026099,-0.025952
4,Cardboard1,0.7,-0.584556,-0.582479,-0.580313,-0.578093,-0.575820,-0.573494,-0.571121,-0.568704,...,-0.206089,-0.205748,-0.205446,-0.205178,-0.204942,-0.204734,-0.204551,-0.204390,-0.204252,-0.204137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
445,Rock1,3.2,-0.009634,-0.009217,-0.008780,-0.008344,-0.007914,-0.007509,-0.007145,-0.006834,...,0.176384,0.176582,0.176750,0.176892,0.177013,0.177115,0.177200,0.177272,0.177332,0.177380
446,Rock1,3.2,-0.016928,-0.016365,-0.015864,-0.015433,-0.015058,-0.014742,-0.014487,-0.014291,...,0.165443,0.165632,0.165795,0.165935,0.166054,0.166157,0.166245,0.166320,0.166384,0.166435
447,Rock1,3.2,-0.998589,-0.999041,-0.999464,-0.999856,-1.000226,-1.000580,-1.000924,-1.001263,...,-0.993987,-0.993959,-0.993938,-0.993921,-0.993910,-0.993901,-0.993896,-0.993893,-0.993891,-0.993891
448,Rock1,3.2,-1.014564,-1.015152,-1.015591,-1.015912,-1.016159,-1.016373,-1.016584,-1.016819,...,-1.012611,-1.012574,-1.012541,-1.012513,-1.012488,-1.012466,-1.012446,-1.012430,-1.012415,-1.012403


In [9]:
e_train = e_data[(e_data["names"]!="Cardboard1" )&(e_data["names"]!="Ceramic1" )&(e_data["names"]!="Glass1")&(e_data["names"]!="Plastic1") &(e_data["names"]!="Rock1")].reset_index(drop=True)
e_train

,names,Density,spectrum_0,spectrum_1,spectrum_2,spectrum_3,spectrum_4,spectrum_5,spectrum_6,spectrum_7,...,spectrum_321,spectrum_322,spectrum_323,spectrum_324,spectrum_325,spectrum_326,spectrum_327,spectrum_328,spectrum_329,spectrum_330
0,Cardboard,0.7,0.058279,0.063007,0.067694,0.072298,0.076858,0.081422,0.086024,0.090695,...,0.557489,0.557749,0.557980,0.558187,0.558372,0.558535,0.558680,0.558808,0.558920,0.559013
1,Cardboard,0.7,0.107668,0.112021,0.116560,0.121207,0.125941,0.130761,0.135658,0.140626,...,0.610426,0.610694,0.610936,0.611154,0.611349,0.611523,0.611679,0.611817,0.611937,0.612038
2,Cardboard,0.7,0.091189,0.095456,0.099860,0.104353,0.108945,0.113654,0.118484,0.123433,...,0.588477,0.588641,0.588790,0.588924,0.589045,0.589153,0.589249,0.589335,0.589409,0.589472
3,Cardboard,0.7,0.111236,0.114431,0.117930,0.121694,0.125711,0.129980,0.134483,0.139196,...,0.611844,0.612121,0.612367,0.612586,0.612779,0.612950,0.613102,0.613235,0.613350,0.613446
4,Cardboard,0.7,0.048949,0.051882,0.055263,0.058999,0.063017,0.067271,0.071713,0.076300,...,0.530075,0.530425,0.530738,0.531018,0.531267,0.531489,0.531685,0.531859,0.532009,0.532135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,Wood,0.8,-0.020559,-0.020252,-0.019827,-0.019304,-0.018699,-0.018044,-0.017370,-0.016704,...,0.135658,0.136021,0.136337,0.136612,0.136852,0.137061,0.137242,0.137398,0.137532,0.137642
806,Wood,0.8,-0.141702,-0.141929,-0.141998,-0.141924,-0.141726,-0.141434,-0.141075,-0.140675,...,-0.008360,-0.008045,-0.007770,-0.007532,-0.007325,-0.007146,-0.006990,-0.006856,-0.006743,-0.006649
807,Wood,0.8,-0.969075,-0.969072,-0.969058,-0.969057,-0.969083,-0.969153,-0.969274,-0.969449,...,-0.960065,-0.959989,-0.959924,-0.959869,-0.959823,-0.959784,-0.959750,-0.959722,-0.959699,-0.959680
808,Wood,0.8,-1.010783,-1.010703,-1.010686,-1.010730,-1.010826,-1.010971,-1.011161,-1.011389,...,-1.012643,-1.012611,-1.012586,-1.012566,-1.012551,-1.012540,-1.012531,-1.012525,-1.012520,-1.012517


In [10]:
e_test_name = e_test["names"].to_numpy()
e_test = e_test.drop(columns = ["names"])
e_train = e_train.drop(columns = ["names"])

In [11]:
e_test.shape[0]

450

In [12]:
e_test_y = e_test["Density"]
e_test_x = e_test.drop(columns = ["Density"])

In [13]:
e_train_y = e_train["Density"]
e_train_x = e_train.drop(columns = ["Density"])

In [14]:
x_tra, x_val, y_tra, y_val = train_test_split(e_train_x, e_train_y, test_size=0.15, random_state=50)

In [15]:
train_size = x_tra.shape[0]

In [16]:
# Define the prior weight distribution as Normal of mean=0 and stddev=1.
# Note that, in this example, the we prior distribution is not trainable,
# as we fix its parameters.
def prior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    prior_model = keras.Sequential(
        [
            tfp.layers.DistributionLambda(
                lambda t: tfp.distributions.MultivariateNormalDiag(
                    loc=tf.zeros(n), scale_diag=tf.ones(n)
                )
            )
        ]
    )
    return prior_model


# Define variational posterior weight distribution as multivariate Gaussian.
# Note that the learnable parameters for this distribution are the means,
# variances, and covariances.
def posterior(kernel_size, bias_size, dtype=None):
    n = kernel_size + bias_size
    posterior_model = keras.Sequential(
        [
            tfp.layers.VariableLayer(
                tfp.layers.MultivariateNormalTriL.params_size(n), dtype=dtype
            ),
            tfp.layers.MultivariateNormalTriL(n),
        ]
    )
    return posterior_model

In [17]:
BATCH_SIZE = 16
EPOCHS = 10
learning_rate = 0.0001
encoding_dim = 331

'''def bnn():
    model = Sequential()
    model.add(tfp.layers.DenseVariational(
            units=encoding_dim,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        ))
    model.add(tfp.layers.DenseVariational(
            units=512,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        ))
    model.add(tfp.layers.DenseVariational(
            units=128,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        ))
    model.add(tfp.layers.DenseVariational(
            units=32,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        ))
    model.add(tfp.layers.DenseVariational(
            units=16,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        ))
    
    model.add(Dense(2))
    model.add(tfp.layers.IndependentNormal(1))
    return model'''
def bnn():
    model = Sequential()
    model.add(tfp.layers.DenseVariational(
            units=2,
            make_prior_fn=prior,
            make_posterior_fn=posterior,
            kl_weight=1 / train_size,
            activation="sigmoid",
        ))
    
    model.add(Dense(2))
    model.add(tfp.layers.IndependentNormal(1))

    #model.add(tfp.layers.IndependentNormal(1))
    return model

In [18]:
def negative_loglikelihood(targets, estimated_distribution):
    return -estimated_distribution.log_prob(targets)

In [19]:
model = bnn()
#loss = keras.losses.MeanSquaredError()
model.compile(
    optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate),
    loss=negative_loglikelihood,
    metrics=[keras.metrics.RootMeanSquaredError()],
    )

2023-02-23 12:47:59.614285: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-23 12:47:59.614321: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-23 12:47:59.614394: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (WH-330-3053-R05): /proc/driver/nvidia/version does not exist
2023-02-23 12:47:59.615196: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.2, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True

In [21]:
callbacks = [
    EarlyStoppingByLossVal(monitor='val_loss', value=0.1, verbose=1),
    # EarlyStopping(monitor='val_loss', patience=2, verbose=0),
    ModelCheckpoint("" ,monitor='val_loss', save_best_only=True, verbose=0),
]

In [22]:
model.fit(x_tra, y_tra, epochs=EPOCHS, validation_data=(x_val, y_val))

Epoch 1/10


Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.


22/22 [==============================] - 2s 30ms/step - loss: 119.1387 - root_mean_squared_error: 3.7062 - val_loss: 127.4458 - val_root_mean_squared_error: 3.3347
Epoch 2/10
22/22 [==============================] - 0s 16ms/step - loss: 112.0738 - root_mean_squared_error: 3.6385 - val_loss: 74.8342 - val_root_mean_squared_error: 3.3155
Epoch 3/10
22/22 [==============================] - 0s 14ms/step - loss: 88.9131 - root_mean_squared_error: 3.6246 - val_loss: 156.6958 - val_root_mean_squared_error: 3.2591
Epoch 4/10
22/22 [==============================] - 0s 14ms/step - loss: 130.9409 - root_mean_squared_error: 3.6058 - val_loss: 157.7437 - val_root_mean_squared_error: 3.2874
Epoch 5/10
22/22 [==============================] - 0s 14ms/step - loss: 137.4406 - root_mean_squared_error: 3.5834 - val_loss: 64.3728 - val_root_mean_squared_error: 3.2454
Epoch 6/10
22/22 [==============================] - 0s 13ms/step - loss: 148.6387 - root_mean_squared_error: 3.7106 - val_loss: 124.9937 - 

In [23]:
model.evaluate(x_tra, y_tra)

22/22 [==============================] - 0s 4ms/step - loss: 133.0158 - root_mean_squared_error: 3.6566


[133.0158233642578, 3.6565704345703125]

In [24]:
line_test = []

prediction_distribution = model.predict(e_test_x)
prediction_mean = prediction_distribution.mean().tolist()
prediction_stdv = prediction_distribution.std()

upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()


15/15 [==============================] - 0s 3ms/step


In [25]:
prediction_distribution

array([[-0.7264672 ],
       [ 0.00480903],
       [-0.9867685 ],
       [-0.66435623],
       [-1.5992937 ],
       [-1.2361368 ],
       [-1.0246941 ],
       [-0.42664215],
       [-0.572362  ],
       [-0.88788855],
       [ 0.97971284],
       [ 0.22859713],
       [-1.3207462 ],
       [-0.6191354 ],
       [-1.044941  ],
       [-0.84055835],
       [-0.8427218 ],
       [-0.8290068 ],
       [ 0.1231644 ],
       [ 0.6092348 ],
       [ 0.37591925],
       [-0.75291663],
       [ 0.06488949],
       [-0.6777403 ],
       [-0.9834964 ],
       [-0.7551162 ],
       [-0.73890024],
       [-1.6120075 ],
       [-0.6192753 ],
       [-0.14268184],
       [-1.1889505 ],
       [-0.91541654],
       [-1.217108  ],
       [-0.6585127 ],
       [-0.47100437],
       [-0.57958555],
       [-0.63335526],
       [-0.47813305],
       [-0.39663145],
       [-0.77461976],
       [-0.7583347 ],
       [-0.56660396],
       [-1.3778598 ],
       [-0.9484024 ],
       [-1.1610663 ],
       [-0

In [26]:
prediction_mean

-0.3962485194206238

In [27]:
for idx in range(len(e_test_name)):
    line_test.append(e_test_name[idx] + ": " + f"Prediction mean: {round(prediction_mean[idx][0], 2)} - stddev: {round(prediction_stdv[idx][0], 2)} - 95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}] - Actual: {e_test_y.to_numpy()[i]}")

TypeError: 'float' object is not subscriptable

In [ ]:
'''# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()

for idx in range(e_test_x.shape[0]):
    print(
        e_test_name[idx],
        f"Prediction mean: {round(prediction_mean[idx][0], 2)}, "
        f"stddev: {round(prediction_stdv[idx][0], 2)}, "
        f"95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}]"
        f" - Actual: {e_test_y[idx]}"
    )'''


In [ ]:
line_test = []

prediction_distribution = model.predict(e_test_x)
prediction_mean = prediction_distribution.mean().numpy().tolist()
prediction_stdv = prediction_distribution.stddev().numpy()

# The 95% CI is computed as mean ± (1.96 * stdv)
upper = (prediction_mean + (1.96 * prediction_stdv)).tolist()
lower = (prediction_mean - (1.96 * prediction_stdv)).tolist()
prediction_stdv = prediction_stdv.tolist()


for i in range(len(pred_test)):
    line_test.append(e_test_name[i] + ": " + f"Prediction mean: {round(prediction_mean[idx][0], 2)} - stddev: {round(prediction_stdv[idx][0], 2)} - 95% CI: [{round(upper[idx][0], 2)} - {round(lower[idx][0], 2)}] - Actual: {e_test_y.to_numpy()[i]}")

In [ ]:
with open('bnn_density_prob.txt', 'w') as f:

    for line in line_test:
        f.write(line)
        f.write('\n')
